In [0]:
import base64

def carregar_credenciais_base64(path):
    rows = spark.read.text(path).collect()
    json_txt = "\n".join([row[0] for row in rows])
    return base64.b64encode(json_txt.encode("utf-8")).decode("utf-8")

credentials_base64 = carregar_credenciais_base64("/Volumes/credentials/credentials/bg_key/bg_key.json")

tabelas_bq = [
    "animes_info_bronze",
    "animes_info_raw",
    "comments_info_raw",
    "forum_info_raw"
]

nome_catalogo = "anime_db"

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {nome_catalogo}")

for tabela in tabelas_bq:
    tabela_bq_full = f"analyticsprojects-447201.Anime_DB.{tabela}"
    destino = f"{nome_catalogo}.{tabela}"

    print(f"🔄 Copiando tabela: {tabela_bq_full} → {destino}")
    
    df = spark.read.format("bigquery") \
        .option("table", tabela_bq_full) \
        .option("parentProject", "analyticsprojects-447201") \
        .option("credentials", credentials_base64) \
        .load()

    df.write.mode("overwrite").saveAsTable(destino)

print("✅ Todas as tabelas foram copiadas com sucesso para o catálogo Anime_DB.")